In [2]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install keras

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Input, Flatten
from sklearn.model_selection import train_test_split
from preprocessing_functions import *

2023-06-08 20:32:05.458820: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-08 20:32:05.501994: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-08 20:32:05.502598: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 20:32:06.184175: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
edges = pd.read_csv("elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv")
features = pd.read_csv("elliptic_bitcoin_dataset/elliptic_txs_features.csv",header=None)
classes = pd.read_csv("elliptic_bitcoin_dataset/elliptic_txs_classes.csv")

In [6]:
edges_by_timestep, features_by_timestep, classes_by_timestep = split_by_time_step(edges, features, classes)
features_num = features.iloc[0].shape[0]
features_num

167

In [7]:
adjacency_map = get_adjacency_map(edges_by_timestep[1])
transactions_out = get_transactions_out_with_adjacency_map(adjacency_map, edges_by_timestep[1].iloc[16]['txId1'], max_depth=10, max_size=1000)
transactions_out

{'txId': 233591710,
 'transaction_trees_out': [{'txId': 234439913,
   'transaction_trees_out': [{'txId': 234441421,
     'transaction_trees_out': [{'txId': 233307261,
       'transaction_trees_out': [{'txId': 234472020,
         'transaction_trees_out': [{'txId': 234472023,
           'transaction_trees_out': [{'txId': 232413175,
             'transaction_trees_out': [{'txId': 230556835,
               'transaction_trees_out': []},
              {'txId': 232413172,
               'transaction_trees_out': [{'txId': 19609789,
                 'transaction_trees_out': []},
                {'txId': 232413178,
                 'transaction_trees_out': [{'txId': 230556838,
                   'transaction_trees_out': 'max_depth reached'},
                  {'txId': 232419645,
                   'transaction_trees_out': 'max_depth reached'}]}]}]}]}]}]},
      {'txId': 234441424,
       'transaction_trees_out': [{'txId': 234441429,
         'transaction_trees_out': [{'txId': 234442451,
        

In [8]:
inversed_adjacency_map = get_inversed_adjacency_map(edges_by_timestep[1])
transactions_in = get_transactions_in_with_inversed_adjacency_map(inversed_adjacency_map, edges_by_timestep[1].iloc[16]['txId1'], max_depth=10, max_size=1000)
transactions_in

{'txId': 233591710,
 'transaction_trees_in': [{'txId': 232007412,
   'transaction_trees_in': [{'txId': 232007419,
     'transaction_trees_in': [{'txId': 232686668,
       'transaction_trees_in': [{'txId': 232686665,
         'transaction_trees_in': [{'txId': 183781611,
           'transaction_trees_in': [{'txId': 232651633,
             'transaction_trees_in': []}]},
          {'txId': 232651633, 'transaction_trees_in': []},
          {'txId': 232665894,
           'transaction_trees_in': [{'txId': 232655296,
             'transaction_trees_in': [{'txId': 230473474,
               'transaction_trees_in': []}]}]},
          {'txId': 232655296,
           'transaction_trees_in': [{'txId': 230473474,
             'transaction_trees_in': []}]}]}]}]}]}]}

In [9]:
root_transaction_id = transactions_in['txId']
print(root_transaction_id)
root_transaction = features_by_timestep[1][features_by_timestep[1][0] == root_transaction_id]
root_transaction

233591710


,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
32,233591710,1,-0.166627,-0.184668,-1.201369,-0.12197,-0.043875,-0.113002,-0.061584,-0.157142,...,-0.554681,-0.594691,-0.979074,-0.978556,0.018279,-0.08749,-0.131155,-0.097524,-0.120613,-0.119792


In [10]:
# All the transactions in one timestep appeared on the blockchain within less than three hours between earch other.
# Bitcoin block time is around 10 minutes.
# Lowest ever average block time in one day was about 3.5 minutes.
# Timeseries limieted to 10 transactions before and 10 transactions after the root transaction because preprocessing would take too long.

transactions_timeseries_length = 21
root_transaction_timeseries_index = 10
transactions_timeseries = [[] for i in range(transactions_timeseries_length)]
transactions_timeseries[root_transaction_timeseries_index].append(root_transaction)

In [11]:
transactions_in['transaction_trees_in']

[{'txId': 232007412,
  'transaction_trees_in': [{'txId': 232007419,
    'transaction_trees_in': [{'txId': 232686668,
      'transaction_trees_in': [{'txId': 232686665,
        'transaction_trees_in': [{'txId': 183781611,
          'transaction_trees_in': [{'txId': 232651633,
            'transaction_trees_in': []}]},
         {'txId': 232651633, 'transaction_trees_in': []},
         {'txId': 232665894,
          'transaction_trees_in': [{'txId': 232655296,
            'transaction_trees_in': [{'txId': 230473474,
              'transaction_trees_in': []}]}]},
         {'txId': 232655296,
          'transaction_trees_in': [{'txId': 230473474,
            'transaction_trees_in': []}]}]}]}]}]}]

In [12]:
def add_transaction_trees_in_to_transactions_timeseries(timeseries, transactions_in, features, insertion_start_index):
    if insertion_start_index < 0:
        return  # depth limit reached

    if transactions_in['transaction_trees_in'] == 'max_depth reached' or transactions_in['transaction_trees_in'] == 'max_size reached':
        return

    for tree in transactions_in['transaction_trees_in']:
        tree_features = features[features[0] == tree['txId']]
        timeseries[insertion_start_index].append(tree_features)
        add_transaction_trees_in_to_transactions_timeseries(timeseries, tree, features, insertion_start_index - 1)

In [13]:
add_transaction_trees_in_to_transactions_timeseries(transactions_timeseries, transactions_in, features_by_timestep[1], root_transaction_timeseries_index - 1)
for i in transactions_timeseries:
    print('[', end='')
    for j in i:
        print(f'{j.shape}, ', end='')
    print(']')

[]
[]
[]
[(1, 167), ]
[(1, 167), (1, 167), (1, 167), ]
[(1, 167), (1, 167), (1, 167), (1, 167), ]
[(1, 167), ]
[(1, 167), ]
[(1, 167), ]
[(1, 167), ]
[(1, 167), ]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [14]:
def add_transaction_trees_out_to_transactions_timeseries(timeseries, transactions_out, features, insertion_start_index):
    if insertion_start_index >= transactions_timeseries_length:
        return  # depth limit reached

    if transactions_out['transaction_trees_out'] == 'max_depth reached' or transactions_out['transaction_trees_out'] == 'max_size reached':
        return

    for tree in transactions_out['transaction_trees_out']:
        tree_features = features[features[0] == tree['txId']]
        timeseries[insertion_start_index].append(tree_features)
        add_transaction_trees_out_to_transactions_timeseries(timeseries, tree, features, insertion_start_index + 1)

In [15]:
add_transaction_trees_out_to_transactions_timeseries(transactions_timeseries, transactions_out, features, root_transaction_timeseries_index + 1)
for i in transactions_timeseries:
    print('[', end='')
    for j in i:
        print(f'{j.shape}, ', end='')
    print(']')

[]
[]
[]
[(1, 167), ]
[(1, 167), (1, 167), (1, 167), ]
[(1, 167), (1, 167), (1, 167), (1, 167), ]
[(1, 167), ]
[(1, 167), ]
[(1, 167), ]
[(1, 167), ]
[(1, 167), ]
[(1, 167), ]
[(1, 167), (1, 167), ]
[(1, 167), (1, 167), (1, 167), ]
[(1, 167), (1, 167), (1, 167), ]
[(1, 167), (1, 167), (1, 167), ]
[(1, 167), (1, 167), (1, 167), (1, 167), (1, 167), ]
[(1, 167), (1, 167), (1, 167), (1, 167), (1, 167), (1, 167), ]
[(1, 167), (1, 167), (1, 167), (1, 167), (1, 167), (1, 167), (1, 167), ]
[(1, 167), (1, 167), (1, 167), (1, 167), (1, 167), (1, 167), (1, 167), (1, 167), ]
[]


In [16]:
transactions_timeseries_summary = pd.DataFrame(np.zeros((transactions_timeseries_length, features_num)))
for i in range(transactions_timeseries_length):
    if len(transactions_timeseries[i]) == 0:
        continue

    transactions_timeseries_summary.loc[i] = pd.concat(transactions_timeseries[i]).mean(axis=0)
    transactions_timeseries_summary.at[i, 0] = len(transactions_timeseries[i])

transactions_timeseries_summary[20:]

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
def get_transaction_timeseries_summary(features, adjacency_map, inversed_adjacency_map, transaction):
    transaction_id = transaction.iloc[0]
    transactions_in = get_transactions_in_with_inversed_adjacency_map(inversed_adjacency_map, transaction_id, max_depth=10, max_size=1000)
    transactions_out = get_transactions_out_with_adjacency_map(adjacency_map, transaction_id, max_depth=10, max_size=1000)

    transactions_timeseries = [[] for i in range(transactions_timeseries_length)]
    transactions_timeseries[root_transaction_timeseries_index].append(transaction)
    add_transaction_trees_in_to_transactions_timeseries(transactions_timeseries, transactions_in, features, root_transaction_timeseries_index - 1)
    add_transaction_trees_out_to_transactions_timeseries(transactions_timeseries, transactions_out, features, root_transaction_timeseries_index + 1)

    transactions_timeseries_summary = pd.DataFrame(np.zeros((transactions_timeseries_length, features_num)))
    for i in range(transactions_timeseries_length):
        if len(transactions_timeseries[i]) == 0:
            continue
    
        transactions_timeseries_summary.loc[i] = pd.concat(transactions_timeseries[i]).mean(axis=0)
        transactions_timeseries_summary.at[i, 0] = len(transactions_timeseries[i])

    return transactions_timeseries_summary

In [44]:
transaction_summaries = []
for timestep in features_by_timestep:
    adjacency_map = get_adjacency_map(edges_by_timestep[timestep])
    inversed_adjacency_map = get_inversed_adjacency_map(edges_by_timestep[timestep])

    index = 0
    for _, transaction in features_by_timestep[timestep].iterrows():
        if index % 10 == 0:
            print(f'\rtime step: {timestep},\tindex: {index}', end='', flush=True)
        index += 1

        # if index == 100:
        #     break

        transaction_id = transaction.iloc[0]

        # Filter out unknown class
        if classes[classes['txId'] == transaction_id].iloc[0][1] == 'unknown':
            continue

        transactions_timeseries_summary = get_transaction_timeseries_summary(features_by_timestep[timestep], adjacency_map, inversed_adjacency_map, transaction)
        transaction_summaries.append({'id': transaction_id, 'summary': transactions_timeseries_summary})

    print()


time step: 1,	index: 7870
time step: 2,	index: 4540
time step: 3,	index: 6620
time step: 4,	index: 5690
time step: 5,	index: 6800
time step: 6,	index: 4320
time step: 7,	index: 6040
time step: 8,	index: 4450
time step: 9,	index: 4990
time step: 10,	index: 6720
time step: 11,	index: 4290
time step: 12,	index: 2040
time step: 13,	index: 4520
time step: 14,	index: 2020
time step: 15,	index: 3630
time step: 16,	index: 2970
time step: 17,	index: 3380
time step: 18,	index: 1970
time step: 19,	index: 3500
time step: 20,	index: 4290
time step: 21,	index: 3530
time step: 22,	index: 5890
time step: 23,	index: 4160
time step: 24,	index: 4590
time step: 25,	index: 2310
time step: 26,	index: 2520
time step: 27,	index: 1080
time step: 28,	index: 1650
time step: 29,	index: 4270
time step: 30,	index: 2480
time step: 31,	index: 2810
time step: 32,	index: 4520
time step: 33,	index: 3150
time step: 34,	index: 2480
time step: 35,	index: 5500
time step: 36,	index: 6390
time step: 37,	index: 3300
time step:

In [19]:
transaction_summaries[:10]

[{'id': 232438397.0,
  'summary':       0             1             2             3             4    \
  0     0.0  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
  1     7.0  1.000000e+00 -1.610807e-01 -1.846676e-01 -1.201369e+00   
  2     9.0  1.000000e+00 -1.437907e-01 -1.846676e-01 -1.201369e+00   
  3    13.0  1.000000e+00 -1.139864e-01 -1.846676e-01 -1.201369e+00   
  4    20.0  1.000000e+00 -1.352125e-01 -1.844993e-01 -1.145870e+00   
  5    38.0  1.000000e+00 -2.148164e-02 -1.835659e-01 -1.040713e+00   
  6    74.0  1.000000e+00 -7.867578e-02 -1.732335e-01 -9.913716e-01   
  7   114.0  1.000000e+00  1.954233e-01 -1.583442e-01 -1.079660e+00   
  8   138.0  1.000000e+00 -1.918985e-02 -1.673788e-01 -9.319158e-01   
  9   160.0  1.000000e+00 -1.302052e-01 -1.771655e-01 -1.086902e+00   
  10    1.0  1.391847e+06  1.391847e+06  1.391847e+06  1.391847e+06   
  11    1.0  1.000000e+00  1.630532e-01 -1.717250e-01 -6.463761e-01   
  12    1.0  1.000000e+00 -1.676577e-01 -1.84

In [45]:
x = []
y = []

for i in transaction_summaries:
    x.append(i['summary'].to_numpy())
    y.append(classes[classes['txId'] == i['id']].iloc[0][1] == '1')

x = np.array(x)
y = np.array(y)
# y
# x[0]

In [46]:
x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.2, random_state=42)

In [22]:
x_train[0].shape

(21, 167)

In [47]:
len(x_train)

36870

In [48]:
rnn_model = Sequential()
rnn_model.add(SimpleRNN(units=64, input_shape=x[0].shape))
rnn_model.add(Dense(units=32, activation='sigmoid'))
rnn_model.add(Dense(units=1, activation='sigmoid'))
rnn_model.compile(loss=['binary_crossentropy'], optimizer='adam', metrics=['accuracy'])
rnn_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_7 (SimpleRNN)    (None, 64)                14848     
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 16,961
Trainable params: 16,961
Non-trainable params: 0
_________________________________________________________________


In [49]:
rnn_model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
1153/1153 [==============================] - 6s 4ms/step - loss: 0.3110 - accuracy: 0.8996 - val_loss: 0.2883 - val_accuracy: 0.9082
Epoch 2/100
1153/1153 [==============================] - 5s 4ms/step - loss: 0.2482 - accuracy: 0.9290 - val_loss: 0.2446 - val_accuracy: 0.9278
Epoch 3/100
1153/1153 [==============================] - 5s 4ms/step - loss: 0.2300 - accuracy: 0.9328 - val_loss: 0.2380 - val_accuracy: 0.9305
Epoch 4/100
1153/1153 [==============================] - 5s 4ms/step - loss: 0.2269 - accuracy: 0.9257 - val_loss: 0.2144 - val_accuracy: 0.9273
Epoch 5/100
1153/1153 [==============================] - 5s 5ms/step - loss: 0.1999 - accuracy: 0.9337 - val_loss: 0.2112 - val_accuracy: 0.9263
Epoch 6/100
1153/1153 [==============================] - 6s 5ms/step - loss: 0.1969 - accuracy: 0.9340 - val_loss: 0.2040 - val_accuracy: 0.9258
Epoch 7/100
1153/1153 [==============================] - 5s 5ms/step - loss: 0.1901 - accuracy: 0.9337 - val_loss: 0.1919 - val_ac

# OLD STUFF